Author: Gokul Krishna Guruswamy

# Pure Python Implementation

In [1]:
from warnings import filterwarnings

In [2]:
filterwarnings('ignore')

In [3]:
import math
from decimal import *

In [4]:
def vec_sigmoid(z):
    # python precision safe sigmoid function
    return [(1.0 / (1.0 + float(Decimal(-i).exp()))) for i in z]

In [5]:
def mean(x):
    return sum(x) / len(x)

We will use the logistic loss function as it is well suited for logistic regression because it’s convex.

In [6]:
def vec_logistic_loss(y, y_hat):
    p1 = list(map(lambda x: x[0] * math.log(x[1]),  zip(y, y_hat)))
    p2 = list(map(lambda x: (1.0 - x[0]) * math.log(1.0 - x[1]),  zip(y, y_hat)))
    p3 = list(map(lambda x: x[0] + x[1], zip(p1, p2)))
    return -mean(p3)

In [7]:
def matmul(X, Y):
    # matrix multiplication implementation

    xw, xh = len(X[0]), len(X)
    yw, yh = len(Y[0]), len(Y)

    if xw == yh:
        
        res = [[0 for i in range(yw)] for j in range(xh)]
        for i in range(len(X)):
            for j in range(len(Y[0])):
                for k in range(len(Y)):
                    res[i][j] += X[i][k] * Y[k][j]

        return(res)

In [8]:
def transpose(x):
    # matrix transpose
    return [[x[j][i] for j in range(len(x))] for i in range(len(x[0]))]

In [9]:
def list_flatten(x):
    return [item for sublist in x for item in sublist]

In [10]:
def elem_sub(X, C):
    # element wise subtraction for matrix
    return [[item - c for item in sublist] for c, sublist in zip(C, X)]

In [11]:
def get_data(fname):
    # helper function to load data in to an array
    X = []
    y = []
    with open(fname, 'r') as f:
        while True:
            buf = f.readline()
            if not buf:
                break
            d = buf.split(',')
            y.append(int(d[-1]))
            X.append([float(i) for i in d[:-1]])
    return X, y

In [13]:
def fit_logistic_reg(X, y, learning_rate=0.0001, no_epochs=3000):

    m = len(y)
    # initialize weights and bias
    W = [[0.00001] for i in range(len(X[0]))]
    b = 0.1

    for epoch in range(no_epochs):

        # multiplying with weights
        Z = matmul(X, W)
        # adding bias terms
        Z = [[i[0] + b] for i in Z]
        # get log odds
        y_hat = list(map(lambda x: vec_sigmoid(x), Z))
        
        # get logistic loss
        loss = vec_logistic_loss(y, list_flatten(y_hat))
        
        # calculate difference between y and log odds
        dz = list(map(lambda x: [x[1] - x[0]],  zip(y, list_flatten(y_hat))))
        t = matmul(transpose(X), dz)

        # calculating gradients for weights and bias
        dw = list(map(lambda x: [(1.0 / m) * x[0]], t))
        db = sum(list_flatten(dz))
        
        # do gradient descent and update weights & bias
        W = elem_sub(W, list(map(lambda x: x[0] * learning_rate, dw)))
        b = b - learning_rate * db

        if epoch % 100 == 0:
            print(f"loss after {epoch} epoch is: {loss}")

    return W, b

In [14]:
def predict(X, W, b, thres=0.5):

    # given set of observations, slope and intercept,
    # return hard and soft predictions

    Z = matmul(X, W)
    Z = [[i[0] + b] for i in Z]
    y_hat = list(map(lambda x: vec_sigmoid(x), Z))

    return list(map(lambda x: 1 if x > thres else 0,
                    list_flatten(y_hat))), y_hat

In [15]:
import random

In [12]:
X, y = get_data('pima-indians-diabetes.csv')

## test train split

In [16]:
rnd_idx = list(range(len(y)))
random.shuffle(rnd_idx)
split = 0.8

In [17]:
X_train = [X[i] for i in rnd_idx[:int(split * len(y))]]
y_train = [y[i] for i in rnd_idx[:int(split * len(y))]]
X_test = [X[i] for i in rnd_idx[int(split * len(y)):]]
y_test = [y[i] for i in rnd_idx[int(split * len(y)):]]

In [18]:
W, b = fit_logistic_reg(X_train, y_train, no_epochs=5000)

loss after 0 epoch is: 0.7105021272591197
loss after 100 epoch is: 0.6252818961215323
loss after 200 epoch is: 0.610526359858133
loss after 300 epoch is: 0.6008106540497504
loss after 400 epoch is: 0.592628726436815
loss after 500 epoch is: 0.5853021880187478
loss after 600 epoch is: 0.5786227198138221
loss after 700 epoch is: 0.5724875444809688
loss after 800 epoch is: 0.5668279111943508
loss after 900 epoch is: 0.5615908857469761
loss after 1000 epoch is: 0.5567330529730734
loss after 1100 epoch is: 0.5522176360985891
loss after 1200 epoch is: 0.5480128862142688
loss after 1300 epoch is: 0.544091053966279
loss after 1400 epoch is: 0.54042767160324
loss after 1500 epoch is: 0.537001020112731
loss after 1600 epoch is: 0.533791716801823
loss after 1700 epoch is: 0.530782386679928
loss after 1800 epoch is: 0.5279573951050773
loss after 1900 epoch is: 0.5253026268419346
loss after 2000 epoch is: 0.5228053012003111
loss after 2100 epoch is: 0.5204538157709458
loss after 2200 epoch is: 0.51

### we will use metrics from sklearn, to compare test and train performance

In [19]:
from sklearn.metrics import *

In [20]:
def get_metrics(X, y):
    preds, y_hat = predict(X, W, b)
    acc = accuracy_score(y, preds)
    auc = roc_auc_score(y, y_hat)
    print(classification_report(y, preds))
    print(f"Accuracy: {auc}, AUC: {auc}")

In [21]:
print(f"class ratio: {sum(y) / len(y)}")

class ratio: 0.3489583333333333


**We can see that classes are not balanced, hence we will use auc and f1 scores to check the model performance.**

In [22]:
get_metrics(X_train, y_train)

              precision    recall  f1-score   support

           0       0.78      0.90      0.84       403
           1       0.73      0.52      0.61       211

   micro avg       0.77      0.77      0.77       614
   macro avg       0.76      0.71      0.72       614
weighted avg       0.77      0.77      0.76       614

Accuracy: 0.8292074841532111, AUC: 0.8292074841532111


In [23]:
get_metrics(X_test, y_test)

              precision    recall  f1-score   support

           0       0.74      0.86      0.79        97
           1       0.67      0.49      0.57        57

   micro avg       0.72      0.72      0.72       154
   macro avg       0.70      0.67      0.68       154
weighted avg       0.71      0.72      0.71       154

Accuracy: 0.8191354675348164, AUC: 0.8191354675348164


**We can see that the test metrics are on the same level in terms of accuracy, AUC, f1-score etc. I have discussed below breifly how to avoid overfitting.**

# scikit-learn

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
model = LogisticRegression(max_iter=3000)

In [26]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=3000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [27]:
def get_metrics_sk(X, y):
    preds = model.predict(X)
    y_hat = model.predict_proba(X)
    acc = accuracy_score(y, preds)
    auc = roc_auc_score(y, [i[1] for i in y_hat])
    print(classification_report(y, preds))
    print(f"Accuracy: {acc}, AUC: {auc}")

In [28]:
get_metrics_sk(X_train, y_train)

              precision    recall  f1-score   support

           0       0.79      0.91      0.85       403
           1       0.76      0.55      0.64       211

   micro avg       0.79      0.79      0.79       614
   macro avg       0.78      0.73      0.74       614
weighted avg       0.78      0.79      0.77       614

Accuracy: 0.7850162866449512, AUC: 0.8335352157397717


In [29]:
get_metrics_sk(X_test, y_test)

              precision    recall  f1-score   support

           0       0.73      0.86      0.79        97
           1       0.66      0.47      0.55        57

   micro avg       0.71      0.71      0.71       154
   macro avg       0.70      0.66      0.67       154
weighted avg       0.71      0.71      0.70       154

Accuracy: 0.7142857142857143, AUC: 0.8260083197684934


**The sklearn model with default settings seem to be performing poorly when compared with the pure python implementation, as indicated by the values of the above metrics, we could do grid search for hyperparameters like regularization, class_weight, penely etc, to achive the best model possible**

# We can handle overfitting in 2 ways:

1. *Early Stopping*: While training the model, we'll monitor the values of some loss metric, by checking it every few iterations. Comparing these values for the validation and training data set will let us know when the model has started overfitting. That will be our cue to stop training.

2. *Regularization*: We can also combat overfitting by using regularization. In this technique, we add an extra term to the cost function, which ends up penalising certain parameter configurations more than others. Examples of this are L-1 regularization and L-2 regularization.

In general, the cost function under L-p regularization:

$$E(\theta,D) = L(\theta,D) + \lambda R(\theta)$$

where,
- $R(\theta) = \lambda (\sum_{j}|\theta_j|^p)$, and
- $\lambda$ = Parameter that controls how much we regularize